In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.0 MB/s eta 0:00:00


# Create individual CER files and concatenate IDs 5 and 7

In [ ]:
import os
from jiwer import cer
from collections import defaultdict
import re

concatClassNomeAndFiliacaoReference = ''
concatClassNomeAndFiliacaoHypothesis = ''

def read_file(file_path):
    data = defaultdict(str)

    all_parts_for_key4 = []
    stored_numbers = []
    stored_long_letters = []
    stored_short_letters = []

    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split(" ", 1)
            key = int(key)
            value = value.strip('"')

            if key == 4:
                parts = value.split()

                numbers = [part for part in parts if any(char.isdigit() for char in part)]
                long_letters = [part for part in parts if len(part) > 2 and len(part) < 5]
                short_letters = [part for part in parts if len(part) <= 2 and not part.isdigit()]

                all_parts_for_key4.extend(parts)

                if numbers:
                    stored_numbers = numbers
                if long_letters:
                    stored_long_letters = long_letters
                if short_letters:
                    stored_short_letters = short_letters

                if stored_numbers and stored_long_letters and stored_short_letters:
                    value_to_add = " ".join(stored_numbers + stored_long_letters + stored_short_letters)
                    data[key] += value_to_add + ' '

            else:
                data[key] += value + ' '

        if all_parts_for_key4:
            value_to_add = " ".join(all_parts_for_key4)
            if 4 not in data or not data[4]:
                data[4] = value_to_add

    return {k: v.strip() for k, v in data.items()}


main_folder_path = '/content/drive/MyDrive/Tesseract/resultados_pt'
metrics_path = '/content/drive/MyDrive/Tesseract/metricas_permissao_pt'

for subdir in os.listdir(main_folder_path):
    hypothesis_path = os.path.join(main_folder_path, subdir)
    reference_path = os.path.join('/content/drive/MyDrive/reference')
    subdir_path = os.path.join(main_folder_path, subdir)

    if os.path.isdir(subdir_path):
        print(subdir_path)

        output_file_path = os.path.join(metrics_path, subdir, 'cer.txt')

        os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

        reference_file_path = os.path.join(reference_path, f'{subdir}_in.txt')
        hypothesis_file_path = os.path.join(hypothesis_path, 'hipotese.txt')

        if os.path.exists(reference_file_path) and os.path.exists(hypothesis_file_path):
            reference_data = read_file(reference_file_path)
            hypothesis_data = read_file(hypothesis_file_path)

            cer_results = {}
            for key in reference_data.keys():
                reference = reference_data.get(key, "")
                hypothesis = hypothesis_data.get(key, "")
                reference = reference.replace('Ã', 'A')
                hypothesis = hypothesis.replace('  ', ' ')

                if key in [5, 7]:
                    concatClassNomeAndFiliacaoReference += reference + ' '
                    concatClassNomeAndFiliacaoHypothesis += hypothesis + ' '
                if hypothesis and reference:
                 cer_results[key] = cer(reference, hypothesis)

            with open(output_file_path, 'w') as output_file:
                for key, error in cer_results.items():
                    output_file.write(f"{key} {error}\n")

            print(f"The results were saved in {output_file_path}")
        else:
            print(f"Files reference.txt and/or hypothesis.txt not found")


In [ ]:
print(concatClassNomeAndFiliacaoReference)

DEUTNER MENICI PARON PATARO FABIANE YBARRA LAURENTINO LELIO JOICE TAMBOR PERENCIN MIRHAN OIE MARCELL CIPOLLETTA DOTORE YOKO VONK CLAUSER CARRI NICOLI BATELLO ZANARDI RZEPIA SILVEIRA KENITH FELIZALI BOSE DUARTE BRAGUIERI BRADNA VASILC ALYNE WEIDLE RAFACHO FLAMINIO CANTARELI MORAES VILHENA PONTUAL ROSELY CIOTTO NUBERTO GHIR LOPES ANSELMO KIOMY JESSIE RODRIGUES EGIMA MITSUMOTO FURCH TJHIO BAYCER RUDY JUTGLA HAMM HISSASHI RODRIGUES JACYANA HILDE GABOLLA CAUDURO AKKERMAN MOHERDAUI MASSAO LAIS MELLO REZENDE MUGGLER MANO LAUDELIN BERGSTEIN ROLNIK CHIAOTING SALES RISTORI TONIDANDEL MANDALITI EIKA NOVELLE BOLGUERONI KUTNER CA JESUS BARBAROTTI GREGORA DA LUZ IUASSE AYOUB SHIMAZU BUTUGAN ANASTACIO CAR WANDA BALPARDA KRATK REZENDE HAMUD CEDISMONDI VELLUTINI BARRETTO KANO ESDRAS AIELO APPEZZATO SCA MARTINS HIDEO ASSIMO TORTORA HINUY BELLON ERRA MURBACK GERALDELLI SILV PEREIRA SORGINI LISBAO TOALIARI MAKI JANAINA HABI ALVES MACARIO TSUMORI RACY MOURA BIAZOLLI CAMARNEIRO ANELY CASETA CHUO NAILU DA CO

In [ ]:
print(concatClassNomeAndFiliacaoHypothesis)

DEUTNER MENICI PARON PATARO FABIANE YBARRA LAURENTINO LELIO JOIÇE TAMBOR PERENCIN MIRHAN IOIE MARCELL, CIPOLLETTA DOTORE YOKO VONK ICLAUSER CARRI NICOLI BATELLO ZANARDI RZEPIA SILVEIRA KENITH FELIZALI BOSE DUARTE BRAGUTERI BRADNA VASILC ALYNE WEIDLE RAFACHO FLAMINIO CANTARELI MORAES (VILHENA PONTUAL ROSELY CrIoTTO NUBERTO GHIR LOPES BNSELMO KIOMY JESSIE RODRIGUES IEGIMA MITSUMOTO FURCH TJEIO BAYCER RUDY JUTGLA HAMM NISSASHI RODRIGUES JACIANA HILDE GADOL LA CAJDURO ARKERMAN MORERDAUL MASSÃO LAIO MELLO REZENDE MUGGLER MANO LAUDELIN BERGSTEIN ROLNIK CHIAOTING SALES RISTORI TONIDANDEL IMANDALITI EIKA NOVELLE BOLGUERONI KUTNER CA JESUS BARBAROTTI GREGORA DA LUZ JNASSF AVYOÇE SHIMAZU PBPUTUÇAN ANASTACIO CAR HZNDE EBLIPAROL KENTK RESENDE, HAMUD CEDISMONDI VELLOUTINT BARRETTO KANO ESDKRAS ALJELO EPPEZLLATO SCA MARTINS IHIDEO ASSIMO TORTORA HINUY BELLON ERRA MURBACK SERALDELLI SILV PEREIRA SORGINT! LISBLO TOBLIART MART JANAINA MABI ALVES MEBCARTO TSUMORI RACY MOURE BIAZOLLI CAMARNEIRO ANELY CAS

# Count all files in the method and create an overall result

In [ ]:
import os
from collections import defaultdict
import numpy as np

root_path = '/content/drive/MyDrive/GPT/metricas_gpt_mini'

class_values = defaultdict(list)
total_success = defaultdict(int)
total_error = defaultdict(int)

for dir_name in os.listdir(root_path):
    dir_path = os.path.join(root_path, dir_name)

    if dir_name == "Geral" or not os.path.isdir(dir_path):
        continue

    cer_file_path = os.path.join(dir_path, 'cer.txt')

    if os.path.exists(cer_file_path):
        with open(cer_file_path, 'r') as cer_file:
            for line in cer_file:
                class_number, value = line.split()
                class_number = int(class_number)
                value = float(value)

                if value > 0.0:
                    total_error[class_number] += 1
                    print(f"Class {class_number}, file {cer_file_path}: ERROR (value = {value})")
                else:
                    total_success[class_number] += 1
                    print(f"Class {class_number}, file {cer_file_path}: SUCCESS (value = {value})")

                class_values[class_number].append(value)

output_file = os.path.join(root_path, 'Geral', 'cer-geral-0.0.txt')
os.makedirs(os.path.join(root_path, 'Geral'), exist_ok=True)

with open(output_file, 'w') as f:
    for class_number in range(0, 12):
        errors = class_values[class_number]
        total_files = len(errors)

        mean_cer = round(np.mean(errors), 5) if total_files > 0 else 0
        std_cer = round(np.std(errors), 5) if total_files > 0 else 0

        success = total_success[class_number]
        error = total_error[class_number]

        f.write(f'{class_number} total_files {total_files} mean_cer {mean_cer} std_cer {std_cer} success {success} error {error}\n')


# Check the Precision, Recall and f1 score of the concatenation of IDs 5 and 7

In [ ]:
class SROIEScorer:
    def __init__(self):
        self.n_detected_words = 0
        self.n_gt_words = 0
        self.n_match_words = 0
        self.ref = []
        self.pred = []

def add_string(scorer, ref, pred):
    pred_words = list(pred.split())
    ref_words = list(ref.split())
    scorer.n_gt_words += len(ref_words)
    scorer.n_detected_words += len(pred_words)
    for pred_w in pred_words:
        if pred_w in ref_words:
            scorer.n_match_words += 1
            ref_words.remove(pred_w)
    scorer.ref.append(ref)
    scorer.pred.append(pred)

def score(scorer):
    prec = scorer.n_match_words / float(scorer.n_detected_words) * 100
    recall = scorer.n_match_words / float(scorer.n_gt_words) * 100
    f1 = 2 * (prec * recall) / (prec + recall)
    return prec, recall, f1

def result_string(scorer):
    prec, recall, f1 = score(scorer)
    return f"Precision: {prec:.3f} Recall: {recall:.3f} F1: {f1:.3f}"

scorer = SROIEScorer()
add_string(scorer, concatClassNomeAndFiliacaoReference, concatClassNomeAndFiliacaoHypothesis)
print(result_string(scorer))


Precision: 72.408 Recall: 72.833 F1: 72.620
